In [212]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [213]:
import pandas as pd
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.metrics import  r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder

In [214]:
df = pd.read_csv('./drive/MyDrive/train.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [215]:
features_with_nan = [feature for feature in df. columns if df[feature].isnull().sum()>1]

In [216]:
features_with_nan.remove('floor')

In [217]:
for feature in features_with_nan:
    print(feature, np.round(df[feature].isnull().mean()*100, 2), '% missing')

osm_city_nearest_population 0.02 % missing
reform_house_population_1000 5.22 % missing
reform_house_population_500 9.73 % missing
reform_mean_floor_count_1000 5.97 % missing
reform_mean_floor_count_500 10.78 % missing
reform_mean_year_building_1000 5.8 % missing
reform_mean_year_building_500 10.59 % missing
street 0.57 % missing


In [218]:
X_df, y_series = df.drop(['floor', 'per_square_meter_price'], axis=1), df['per_square_meter_price']

In [219]:
numerical_feat=[features for features in df.columns if df[features].dtypes!='O']
print('The number of numerical features are', len(numerical_feat))

The number of numerical features are 70


In [220]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [221]:
df_dropped = X_df.drop(features_with_nan, axis = 1)

In [222]:
cat_features = []
for i in df_dropped.columns:
    if i not in numerical_feat:
        cat_features.append(i)

In [223]:
df_dropped = df_dropped.drop(cat_features, axis = 1)

In [224]:
scaler = StandardScaler()
support_scaled = scaler.fit_transform(df_dropped)

In [225]:
clust = KMeans()

clst_labels = clust.fit_predict(support_scaled)
labels_set = list(set(clst_labels))
print(labels_set)

[0, 1, 2, 3, 4, 5, 6, 7]


In [226]:
clst_labels.shape

(279792,)

In [227]:
X_df['cluster'] = clst_labels

In [228]:
features_nan_num = []
for i in features_with_nan:
  if i in numerical_feat:
    features_nan_num.append(i)

In [229]:
X_df.groupby('cluster')['cluster']

In [230]:
sub_df = X_df[X_df[features_with_nan].isnull()]

In [231]:
for drop in features_nan_num:
  sub_df = X_df[X_df[drop].isnull()]
  for clust in sub_df.groupby('cluster')['cluster'].unique():
    similar_val_list = X_df[X_df['cluster'] == clust[0]][drop]
    if len(similar_val_list) > 1:
      missed_val = similar_val_list.dropna().mean()
      X_df.loc[similar_val_list[similar_val_list.isnull()].index, drop] = missed_val

In [232]:
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279792 entries, 0 to 279791
Data columns (total 76 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   city                                 279792 non-null  object 
 1   id                                   279792 non-null  object 
 2   lat                                  279792 non-null  float64
 3   lng                                  279792 non-null  float64
 4   osm_amenity_points_in_0.001          279792 non-null  int64  
 5   osm_amenity_points_in_0.005          279792 non-null  int64  
 6   osm_amenity_points_in_0.0075         279792 non-null  int64  
 7   osm_amenity_points_in_0.01           279792 non-null  int64  
 8   osm_building_points_in_0.001         279792 non-null  int64  
 9   osm_building_points_in_0.005         279792 non-null  int64  
 10  osm_building_points_in_0.0075        279792 non-null  int64  
 11  osm_building_

In [233]:
features_nan_cat = [feature for feature in X_df. columns if X_df[feature].isnull().sum()>1]

In [234]:
features_nan_cat

['street']

In [235]:
len(X_df['street'].unique())

28842

In [236]:
X_df['street'] = X_df['street'].str[1:]

In [237]:
X_df['street']

0         27289
1         17052
2         16913
3         10148
4          1338
          ...  
279787    11114
279788    28440
279789     6671
279790    17667
279791     5611
Name: street, Length: 279792, dtype: object

In [238]:
X_df['street'].fillna(0)

0         27289
1         17052
2         16913
3         10148
4          1338
          ...  
279787    11114
279788    28440
279789     6671
279790    17667
279791     5611
Name: street, Length: 279792, dtype: object

In [239]:
X_df['street'] = pd.to_numeric(X_df['street'])

In [240]:
X_df['street'].replace({0:X_df['street'].median()})

0         27289.0
1         17052.0
2         16913.0
3         10148.0
4          1338.0
           ...   
279787    11114.0
279788    28440.0
279789     6671.0
279790    17667.0
279791     5611.0
Name: street, Length: 279792, dtype: float64

In [241]:
X_df['street']

0         27289.0
1         17052.0
2         16913.0
3         10148.0
4          1338.0
           ...   
279787    11114.0
279788    28440.0
279789     6671.0
279790    17667.0
279791     5611.0
Name: street, Length: 279792, dtype: float64

### CatBoost

In [242]:
pip install catboost

In [243]:
X_df = X_df.drop(['id','date'], axis=1)

In [244]:
X_df_num = X_df.drop(['region', 'city'], axis=1)

In [245]:
X_df_num = X_df_num.drop(['osm_city_nearest_name'], axis=1)

In [166]:
from catboost import CatBoostRegressor

In [167]:
ctb = CatBoostRegressor(random_seed=17)

In [168]:
train_df, valid_df, y_train, y_valid = train_test_split(X_df_num, y_series, 
                                                                  test_size=0.7, random_state=42)

In [169]:
ctb.fit(train_df, y_train)

Learning rate set to 0.085875
0:	learn: 165015.7314249	total: 44.3ms	remaining: 44.3s
1:	learn: 158834.3633295	total: 84.8ms	remaining: 42.3s
2:	learn: 153232.2160497	total: 126ms	remaining: 41.8s
3:	learn: 148401.6127198	total: 166ms	remaining: 41.4s
4:	learn: 143978.4721452	total: 230ms	remaining: 45.8s
5:	learn: 140063.9117664	total: 272ms	remaining: 45.1s
6:	learn: 136640.1877040	total: 311ms	remaining: 44.1s
7:	learn: 133604.4013688	total: 351ms	remaining: 43.5s
8:	learn: 130933.1606575	total: 400ms	remaining: 44.1s
9:	learn: 128576.8748327	total: 446ms	remaining: 44.2s
10:	learn: 126513.7543105	total: 489ms	remaining: 44s
11:	learn: 124625.1410843	total: 532ms	remaining: 43.8s
12:	learn: 122851.2693669	total: 568ms	remaining: 43.1s
13:	learn: 121228.6803651	total: 613ms	remaining: 43.2s
14:	learn: 119929.5673946	total: 657ms	remaining: 43.2s
15:	learn: 118761.7274259	total: 695ms	remaining: 42.7s
16:	learn: 117693.7871870	total: 737ms	remaining: 42.6s
17:	learn: 116788.5903948	to

In [170]:
import typing

In [171]:
rmse = mean_squared_error(ctb.predict(valid_df), y_valid, squared=False)

In [172]:
rmse

88816.498269252

In [173]:
import joblib

In [174]:
joblib.dump(ctb,"ctb.pkl")#regressor

['ctb.pkl']

In [175]:
test_df = pd.read_csv('./drive/MyDrive/test.csv')

In [176]:
test_df = test_df.drop(['floor'], axis=1)

In [177]:
test_dropped = test_df.drop(features_with_nan, axis = 1)

In [178]:
test_dropped = test_dropped.drop(cat_features, axis = 1)

In [179]:
diff = []
for i in df_dropped.columns:
  if i not in test_dropped.columns:
    diff.append(i)

In [180]:
diff

[]

In [181]:
set()

set()

In [182]:
test_scaled = scaler.transform(test_dropped)

### Нужно будет объединить тест и трэйн

In [187]:
clust = KMeans()

In [188]:
test_clst = clust.fit_predict(test_scaled)


In [189]:
labels = list(set(test_clst))

In [190]:
test_df['cluster'] = test_clst

In [191]:
sub_test = test_df[test_df[features_with_nan].isnull()]

In [192]:
for drop in features_nan_num:
  sub_test = test_df[test_df[drop].isnull()]
  for clust in sub_test.groupby('cluster')['cluster'].unique():
    similar_val_list = test_df[X_df['cluster'] == clust[0]][drop]
    if len(similar_val_list) > 1:
      missed_val = similar_val_list.dropna().mean()
      test_df.loc[similar_val_list[similar_val_list.isnull()].index, drop] = missed_val

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will

In [194]:
test_df['street'] = test_df['street'].str[1:]

In [195]:
test_df['street'].fillna(0)

0        6983
1       29120
2       23731
3       14207
4       20658
        ...  
2969     2650
2970    11114
2971    34005
2972    21753
2973    28997
Name: street, Length: 2974, dtype: object

In [196]:
test_df['street'] = pd.to_numeric(test_df['street'])

In [197]:
test_df['street'].replace({0:test_df['street'].median()})

0        6983.0
1       29120.0
2       23731.0
3       14207.0
4       20658.0
         ...   
2969     2650.0
2970    11114.0
2971    34005.0
2972    21753.0
2973    28997.0
Name: street, Length: 2974, dtype: float64

In [199]:
test_df = test_df.drop(['id','date'], axis=1)
test_df_num = test_df.drop(['region', 'city'], axis=1)
test_df_num = test_df_num.drop(['osm_city_nearest_name'], axis=1)

In [200]:
answer = ctb.predict(test_df_num)

In [202]:
answer.shape

(2974,)

In [203]:
submission = pd.read_csv('./drive/MyDrive/test_submission.csv')

In [207]:
submission['per_square_meter_price'] = answer

In [208]:
submission

,id,per_square_meter_price
0,COL_289284,27529.908167
1,COL_289305,53475.359313
2,COL_289318,54625.658465
3,COL_289354,87061.653513
4,COL_289399,79533.405834
...,...,...
2969,COL_455089,29667.955964
2970,COL_455212,40112.203496
2971,COL_455261,38674.315726
2972,COL_455381,59331.040212


In [209]:
from google.colab import files
submission.to_csv('first.csv') 
files.download('first.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>